### A note on terminology:

You'll notice that there are quite a few similarities between LangChain and LlamaIndex. LlamaIndex can largely be thought of as an extension to LangChain, in some ways - but they moved some of the language around. Let's spend a few moments disambiguating the language.

- `QueryEngine` -> `RetrievalQA`:
  -  `QueryEngine` is just LlamaIndex's way of indicating something is an LLM "chain" on top of a retrieval system
- `OpenAIAgent` vs. `ZeroShotAgent`:
  - The two agents have the same fundamental pattern: Decide which of a list of tools to use to answer a user's query.
  - `OpenAIAgent` (LlamaIndex's primary agent) does not need to rely on an agent excecutor due to the fact that it is leveraging OpenAI's [functional api](https://openai.com/blog/function-calling-and-other-api-updates) which allows the agent to interface "directly" with the tools instead of operating through an intermediary application process.

There is, however, a much large terminological difference when it comes to discussing data.

##### Nodes vs. Documents

As you're aware of from the previous weeks assignments, there's an idea of `documents` in NLP which refers to text objects that exist within a corpus of documents.

LlamaIndex takes this a step further and reclassifies `documents` as `nodes`. Confusingly, it refers to the `Source Document` as simply `Documents`.

The `Document` -> `node` structure is, almost exactly, equivalent to the `Source Document` -> `Document` structure found in LangChain - but the new terminology comes with some clarity about different structure-indices. 

We won't be leveraging those structured indicies today, but we will be leveraging a "benefit" of the `node` structure that exists as a default in LlamaIndex, which is the ability to quickly filter nodes based on their metadata.

![image](https://i.imgur.com/B1QDjs5.png)

# Creating a more robust RAQA system using LlamaIndex

We'll be putting together a system for querying both qualitative and quantitative data using LlamaIndex. 

To stick to a theme, we'll continue to use BarbenHeimer data as our base - but this can, and should, be extended to other topics/domains.

# Build 🏗️
There are 3 main tasks in this notebook:

- Create a Qualitative VectorStore query engine
- Create a quantitative NLtoSQL query engine
- Combine the two using LlamaIndex's OpenAI agent framework.

# Ship 🚢
Create an host a Gradio or Chainlit application to serve your project on Hugging Face spaces.

# Share 🚀
Make a social media post about your final application and tag @AIMakerspace

### BOILERPLATE

This is only relevant when running the code in a Jupyter Notebook.

In [1]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Primary Dependencies and Context Setting

#### Dependencies and OpenAI API key setting

First of all, we'll need our primary libraries - and to set up our OpenAI API key.

In [2]:
%pip install -U -q openai==0.27.8 llama-index==0.8.40 nltk==3.8.1 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import llama_index

llama_index.__version__

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


'0.8.40'

In [4]:
import os
import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
# os.environ["WANDB_API_KEY"] = getpass.getpass("WandB API Key: ")

In [6]:
from llama_index import set_global_handler

set_global_handler("wandb", run_args={"project": "llamaindex-demo-v1"})
wandb_callback = llama_index.global_handler

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Streaming LlamaIndex events to W&B at https://wandb.ai/sumush/llamaindex-demo-v1/runs/wnk8sgjh
wandb: `WandbCallbackHandler` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `llamaindex`.


#### Context Setting

Now, LlamaIndex has the ability to set `ServiceContext`. You can think of this as a config file of sorts. The basic idea here is that we use this to establish some core properties and then can pass it to various services. 

While we could set this up as a global context, we're going to leave it as `ServiceContext` so we can see where it's applied.

We'll set a few significant contexts:

- `chunk_size` - this is what it says on the tin
- `llm` - this is where we can set what model we wish to use as our primary LLM when we're making `QueryEngine`s and more
- `embed_model` - this will help us keep our embedding model consistent across use cases


We'll also create some resources we're going to keep consistent across all of our indices today.

- `text_splitter` - This is what we'll use to split our text, feel free to experiment here
- `SimpleNodeParser` - This is what will work in tandem with the `text_splitter` to parse our full sized documents into nodes.

In [7]:
from llama_index import ServiceContext
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()### YOUR CODE HERE
chunk_size = 500### YOUR CODE HERE
llm = OpenAI(
    temperature=0,### YOUR CODE HERE
    model="gpt-3.5-turbo-0613",### YOUR CODE HERE
    streaming=True
)

service_context = ServiceContext.from_defaults(
    llm=llm,### YOUR CODE HERE
    chunk_size=chunk_size,### YOUR CODE HERE
    embed_model=embed_model### YOUR CODE HERE
)

text_splitter = TokenTextSplitter(
    chunk_size=chunk_size### YOUR CODE HERE
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter### YOUR CODE HERE
)

### BarbenHeimer Wikipedia Retrieval Tool

Now we can get to work creating our semantic `QueryEngine`!

We'll follow a similar pattern as we did with LangChain here - and the first step (as always) is to get dependencies.

In [8]:
%pip install -U -q tiktoken==0.4.0 sentence-transformers==2.2.2 pydantic==1.10.11

Note: you may need to restart the kernel to use updated packages.


#### GPTIndex

We'll be using [GPTIndex](https://gpt-index.readthedocs.io/en/v0.6.2/reference/indices/vector_store.html) as our `VectorStore` today!

It works in a similar fashion to tools like Pinecone, Weaveate, and more - but it's locally hosted and will serve our purposes fine. 

Also the `GPTIndex` is integrated with WandB for index versioning.

You'll also notice the return of `OpenAIEmbedding()`, which is the embeddings model we'll be leveraging. Of course, this is using the `ada` model under the hood - and already comes equipped with in-memory caching.

You'll notice we can pass our `service_context` into our `VectorStoreIndex`!

In [9]:
from llama_index import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents([], service_context=service_context)

wandb: Logged trace tree to W&B.


In [10]:
%pip install -U -q wikipedia

Note: you may need to restart the kernel to use updated packages.


Essentially the same as the LangChain example - we're just going to be pulling information straight from Wikipedia using the built in `WikipediaReader`.

Setting `auto_suggest=False` ensures we run into fewer auto-correct based errors.

In [11]:
from llama_index.readers.wikipedia import WikipediaReader

movie_list = [
    "Barbie (film)", 
    "Oppenheimer (film)"
]

wiki_docs = WikipediaReader().load_data(
    pages=movie_list,
    auto_suggest=False
    ### YOUR CODE HERE
)

#### Node Construction

Now we will loop through our documents and metadata and construct nodes (associated with particular metadata for easy filtration later).

We're using the `node_parser` we created at the top of the Notebook.

In [12]:
for movie, wiki_doc in zip(movie_list, wiki_docs):
    nodes = node_parser.get_nodes_from_documents([wiki_doc])
    for node in nodes:
        node.metadata = {"title" : movie}
    index.insert_nodes(nodes)

In [13]:
wandb_callback.persist_index(index, index_name="wiki-index")

wandb: Adding directory to artifact (/Users/ukizhake/Documents/LLM-Ops-Cohort-2-main-week3-thurs/Week 3/Thursday/wandb/run-20231006_233731-wnk8sgjh/files/storage)... Done. 0.0s


In [14]:
from llama_index import load_index_from_storage

storage_context = wandb_callback.load_storage_context(
    artifact_url="sumush/llamaindex-demo-v1/wiki-index:v0"### YOUR ARTIFACT URL HERE
)

index = load_index_from_storage(storage_context, service_context=service_context)

wandb:   4 of 4 files downloaded.  


INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Failed to log trace tree to W&B: list index out of range


In [15]:
wandb_callback.load_storage_context(artifact_url="sumush/llamaindex-demo-v1/wiki-index:v0")

wandb:   4 of 4 files downloaded.  


StorageContext(docstore=<llama_index.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x28dfe1690>, index_store=<llama_index.storage.index_store.simple_index_store.SimpleIndexStore object at 0x2abdee0d0>, vector_store=<llama_index.vector_stores.simple.SimpleVectorStore object at 0x2abdc8190>, graph_store=<llama_index.graph_stores.simple.SimpleGraphStore object at 0x2abdeefd0>)

#### Auto Retriever Functional Tool

This tool will leverage OpenAI's functional endpoint to select the correct metadata filter and query the filtered index - only looking at nodes with the desired metadata.

A simplified diagram: ![image](https://i.imgur.com/AICDPav.png)

First, we need to create our `VectoreStoreInfo` object which will hold all the relevant metadata we need for each component (in this case title metadata).

Notice that you need to include it in a text list.

In [16]:
from llama_index.tools import FunctionTool
from llama_index.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3



Now we'll create our base PyDantic object that we can use to ensure compatability with our application layer. This verifies that the response from the OpenAI endpoint conforms to this schema.

In [17]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

Now we can build our function that we will use to query the functional endpoint.

>The `docstring` is important to the functionality of the application.

In [18]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever, service_context=service_context)

    response = query_engine.query(query)
    return str(response)

Now we need to wrap our system in a tool in order to integrate it into the larger application.

Source Code Here:
- [`FunctionTool`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/tools/function_tool.py#L21)

In [19]:
from typing import Callable 
vector_store_info = VectorStoreInfo(
    content_info="semantic information about movies",
    metadata_info=[MetadataInfo(
        name="title",
        type="str",
        description="title of the movie, one of [Barbie (film), Oppenheimer (film)]",
        # to_openai_function=
    )]
)
description = f"""\
Use this tool to look up semantic information about films.
The vector database schema is given below:
{vector_store_info.json()}
"""

auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn,### YOUR CODE HERE
    name="semantic-film-info",### YOUR CODE HERE
    description=description,### YOUR CODE HERE
    fn_schema=AutoRetrieveModel,### YOUR CODE HERE
    # tool_metadata=vector_store_info.metadata_info,
)

All that's left to do is attach the tool to an OpenAIAgent and let it rip!

Source Code Here:
- [`OpenAIAgent`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/agent/openai_agent.py#L361)

In [20]:
from llama_index.agent import OpenAIAgent

agent = OpenAIAgent.from_tools(
    tools=[
        auto_retrieve_tool### YOUR CODE HERE
    ],
)

In [21]:
agent.chat("what is the plot")

wandb: Logged trace tree to W&B.


AgentChatResponse(response='Sure, could you please provide me with the name of the film you want to know the plot of?', sources=[], source_nodes=[])

In [22]:
response = agent.chat("Tell me what happens (briefly) in the Barbie movie.")
print(str(response))

wandb: Logged trace tree to W&B.


I apologize, but I couldn't find specific plot details for the Barbie movie. However, I can provide you with some general information. The Barbie movie is a live-action film directed by Greta Gerwig and stars Margot Robbie as Barbie and Ryan Gosling as Ken. It revolves around Barbie and Ken's journey of self-discovery after experiencing an existential crisis. The film received critical acclaim and became the highest-grossing film of its release year.


### BarbenHeimer SQL Tool

We'll walk through the steps of creating a natural language to SQL system in the following section.

> NOTICE: This does not have parsing on the inputs or intermediary calls to ensure that users are using safe SQL queries. Use this with caution in a production environment without adding specific guardrails from either side of the application.

In [23]:
%pip install -q -U sqlalchemy pandas

Note: you may need to restart the kernel to use updated packages.


The next few steps should be largely straightforward, we'll want to:

1. Read in our `.csv` files into `pd.DataFrame` objects
2. Create an in-memory `sqlite` powered `sqlalchemy` engine
3. Cast our `pd.DataFrame` objects to the SQL engine
4. Create an `SQLDatabase` object through LlamaIndex
5. Use that to create a `QueryEngineTool` that we can interact with through the `NLSQLTableQueryEngine`!

If you get stuck, please consult the documentation.

#### Read `.csv` Into Pandas

In [24]:
import pandas as pd

barbie_df = pd.read_csv("./barbie_data/barbie.csv")
oppenheimer_df = pd.read_csv("./oppenheimer_data/oppenheimer.csv")

In [25]:
barbie_df

,Unnamed: 0,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,0,21 July 2023,LoveofLegacy,6.0,"Beautiful film, but so preachy\n","Margot does the best with what she's given, bu...",/review/rw9199947/?ref_=tt_urv
1,1,22 July 2023,imseeg,7.0,3 reasons FOR seeing it and 1 reason AGAINST.\n,The first reason to go see it:,/review/rw9199947/?ref_=tt_urv
2,2,22 July 2023,Natcat87,6.0,Too heavy handed\n,"As a woman that grew up with Barbie, I was ver...",/review/rw9199947/?ref_=tt_urv
3,3,31 July 2023,ramair350,10.0,"As a guy I felt some discomfort, and that's o...",As much as it pains me to give a movie called ...,/review/rw9199947/?ref_=tt_urv
4,4,24 July 2023,heatherhilgers,9.0,A Technicolor Dream\n,"Wow, this movie was a love letter to cinema. F...",/review/rw9199947/?ref_=tt_urv
...,...,...,...,...,...,...,...
120,120,19 July 2023,mckai-89546,9.0,Best for Ryan Gosling Gives a really great pe...,I am a man but watched this movie it's awesome...,/review/rw9199947/?ref_=tt_urv
121,121,3 August 2023,PennyReviews,6.0,Good Enough\n,Barbie is a fun summery movie for all ages.,/review/rw9199947/?ref_=tt_urv
122,122,30 July 2023,walfordior,9.0,This is more than just a 'lighthearted comedy'\n,"Barbie is no longer just a toy; rather, this i...",/review/rw9199947/?ref_=tt_urv
123,123,5 August 2023,HuskerNation1,7.0,"Funny And Good Cast, But Not This ""Great"" Mov...","I wasn't dying to see this, but I felt like I ...",/review/rw9199947/?ref_=tt_urv


#### Create SQLAlchemy engine with SQLite

In [26]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:")

#### Convert `pd.DataFrame` to SQL tables

In [27]:
barbie_df.to_sql(
    "barbie",
    engine
)

125

In [28]:
oppenheimer_df.to_sql(
    "oppenheimer",
    engine
)

150

#### Construct a `SQLDatabase` index

Source Code Here:
- [`SQLDatabase`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/langchain_helpers/sql_wrapper.py#L9)

In [29]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(
    engine=engine,
    include_tables=[
        "barbie",### YOUR CODE HERE
        "oppenheimer",### YOUR CODE HER
    ]
)

#### Create the NLSQLTableQueryEngine interface for all added SQL tables

Source Code Here:
- [`NLSQLTableQueryEngine`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/indices/struct_store/sql_query.py#L75C1-L75C1)

In [30]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,### YOUR CODE HERE
    tables=[
        "barbie",### YOUR CODE HERE,
        "oppenheimer",### YOUR CODE HER
    ],### YOUR CODE HERE, 
    service_context=service_context,### YOUR CODE HERE
)

#### Wrap It All Up in a `QueryEngineTool`

You'll want to ensure you have a descriptive...description. 

An example is provided here:

```
"Useful for translating a natural language query into a SQL query over a table containing: "
"barbie, containing information related to reviews of the Barbie movie"
"oppenheimer, containing information related to reviews of the Oppenheimer movie"
```

Sorce Code Here: 

- [`QueryEngineTool`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/tools/query_engine.py#L13)

In [31]:
from llama_index.tools.query_engine import QueryEngineTool,ToolMetadata

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,### YOUR CODE HERE
    name="sql-query",### YOUR CODE HERE
    description=(   
      "Useful for translating a natural language query into a SQL query over a table containing: "
      "barbie, containing information related to reviews of the Barbie movie"
      "oppenheimer, containing information related to reviews of the Oppenheimer movie"
        ### YOUR CODE HERE
    ),
)

In [32]:
agent = OpenAIAgent.from_tools(
    tools=[
        [sql_tool]### YOUR CODE HERE
    ],
)

In [42]:

# response = agent.chat("What is the average rating of the two films?")

In [34]:
print(str(response))

I apologize, but I couldn't find specific plot details for the Barbie movie. However, I can provide you with some general information. The Barbie movie is a live-action film directed by Greta Gerwig and stars Margot Robbie as Barbie and Ryan Gosling as Ken. It revolves around Barbie and Ken's journey of self-discovery after experiencing an existential crisis. The film received critical acclaim and became the highest-grossing film of its release year.


### Combining The Tools Together

Now, we can simple add our tools into the `OpenAIAgent`, and off we go!

In [35]:
barbenheimer_agent = OpenAIAgent.from_tools(
    tools=[
        auto_retrieve_tool,### YOUR CODE HERE
        sql_tool### YOUR CODE HERE
    ],
)

In [36]:
response = barbenheimer_agent.chat("What is the lowest rating of the two films - and can you summarize what the reviewer said?")

INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'barbie' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'oppenheimer' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .
> Table desc str: Table 'barbie' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'oppenheimer' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .


wandb: Logged trace tree to W&B.


In [37]:
print(str(response))

The lowest rating for both films is 3.0. 

For the Barbie movie, one reviewer mentioned that it lacked science and interesting problem-solving. 

For the Oppenheimer movie, one reviewer mentioned that while the production value, cinematography, and acting were good, the movie fell short in some aspects.


In [38]:
response = barbenheimer_agent.chat("How many times do the Barbie reviews mention 'Ken', and what is a summary of his character in the Barbie movie?")

INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Table 'barbie' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'oppenheimer' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .
> Table desc str: Table 'barbie' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'oppenheimer' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .


wandb: Logged trace tree to W&B.


In [39]:
print(str(response))

The Barbie reviews mention Ken a total of 30 times.

In the Barbie movie, Ken is portrayed as a supporting character. While the specific details of his character may vary depending on the individual reviews, Ken is typically depicted as Barbie's love interest and partner. He is often described as charming, handsome, and supportive of Barbie throughout her journey. Ken's character adds a romantic element to the story and contributes to the overall narrative of the film.


In [40]:
wandb_callback.finish()